<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/cartinaEuropa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

# Percorso del file
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetCartinaEuropa.csv")


def generate_minimal_smoking_map():
  # Controllo per il nome del path
    if not os.path.exists(file_path):
        print(f"Errore: non c'è '{file_path}'.")
        return

    try:
        # Carica la tabella
        df = pd.read_csv(file_path)

        # Filtraggio per l'anno 2023 - Totale (T) - Percentuale (PC_POP)
        mask = (
            (df['TIME_PERIOD'] == 2023) &
            (df['sex'] == 'T') &
            (df['unit'] == 'PC_POP')
        )
        df_filtered = df[mask].copy()

        # Mapping ISO2 -> ISO3 per Plotly?!?!
        iso_mapping = {
            'AT': 'AUT', 'BE': 'BEL', 'BG': 'BGR', 'CY': 'CYP', 'CZ': 'CZE',
            'DE': 'DEU', 'DK': 'DNK', 'EE': 'EST', 'EL': 'GRC', 'ES': 'ESP',
            'FI': 'FIN', 'FR': 'FRA', 'HR': 'HRV', 'HU': 'HUN', 'IE': 'IRL',
            'IT': 'ITA', 'LT': 'LTU', 'LU': 'LUX', 'LV': 'LVA', 'MT': 'MLT',
            'NL': 'NLD', 'PL': 'POL', 'PT': 'PRT', 'RO': 'ROU', 'SE': 'SWE',
            'SI': 'SVN', 'SK': 'SVK', 'IS': 'ISL', 'NO': 'NOR', 'CH': 'CHE',
            'UK': 'GBR', 'TR': 'TUR', 'RS': 'SRB', 'AL': 'ALB', 'ME': 'MNE',
            'MK': 'MKD', 'LI': 'LIE'
        }

        df_filtered['iso_alpha'] = df_filtered['geo'].map(iso_mapping)

        # Rimuove le righe senza codice ISO o senza il valore almeno sono tutte dello stesso colore
        df_filtered = df_filtered.dropna(subset=['iso_alpha', 'OBS_VALUE'])

        # Serve per la barra colori a lato
        real_min = df_filtered['OBS_VALUE'].min()
        real_max = df_filtered['OBS_VALUE'].max()
        # Valore dummy per la sezione grigia
        dummy_val = real_min - 5

        # Riga "finta" per forzare la sezione grigia nella barra dei colori
        dummy_row = pd.DataFrame({
            'iso_alpha': [None],
            'OBS_VALUE': [dummy_val],
            'geo': ['Dati mancanti']
        })
        df_plot = pd.concat([df_filtered, dummy_row], ignore_index=True)

        # Scala colori inizia con il grigio poi prosegue con i blu
        custom_blues = px.colors.sequential.Blues
        custom_scale = [
            [0.0, "#cccccc"],        # Sezione grigia per i dati mancanti
            [0.1, "#cccccc"],
            [0.1, custom_blues[0]],  # Inizio dei blu
            [1.0, custom_blues[-1]]   # Massimo dei blu
        ]

        # Crea la mappa base
        fig = px.choropleth(
            df_plot,
            locations="iso_alpha",
            color="OBS_VALUE",
            hover_name="geo",
            color_continuous_scale=custom_scale,
            range_color=[dummy_val, real_max],
            scope="europe",
            labels={'OBS_VALUE': '% Fumatori'}
        )

        # Configurazione della mappa
        fig.update_geos(
            showframe=False,
            showcoastlines=True,
            showcountries=True,
            showlakes=False, # Toglie i laghi
            showrivers=False,
            countrycolor="#444", # Colore confini
            landcolor="#cccccc", # Grigio chiaro per i paesi senza dati
            projection_type="natural earth",
            resolution=50,
            # Zoom sull'Europa
            lataxis_range=[32, 72],
            lonaxis_range=[-25, 45]
        )

        #Titolo e sottotitolo!
        fig.update_layout(
            title={
                'text': '<span style="color:#2E4053; font-size:24px; font-weight:bold;">Densità Fumatori in Europa</span><br><span style="color:#95A5A6; font-size:16px;">Dati del 2023 • Percentuale popolazione</span>',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            margin={"r":0,"t":80,"l":0,"b":0},
            showlegend=False,
            coloraxis_colorbar=dict(
                title=dict(
                    text="% Popolazione<br> &nbsp;", # Titolo della barra colori
                    side="top"
                ),
                thickness=20,
                len=0.6,
                x=0.9,
                y=0.45,
                # Testi decenti della barra colori
                tickvals=[dummy_val, real_min, (real_min + real_max)/2, real_max],
                ticktext=["Dati non sufficienti", f"{real_min}%", f"{(real_min + real_max)/2:.0f}%", f"{real_max}%"],
                tickmode="array"
            )
        )

        # Pulizia hover
        fig.update_traces(
            hovertemplate="<b>%{hovertext}</b><br>%{z}%<extra></extra>"
        )

        fig.show()

    except Exception as e:
        print(f"Errore durante l'elaborazione: {e}")

if __name__ == "__main__":
    generate_minimal_smoking_map()